### Merge all grant votes datasets together
- Merge all voting datasets into one dataframe
- Drop invalid wallets
- Convert date to common format - timestamp
- Convert amounts to usd (using rates from the last day of round for simplicity)
- Save to file

In [1]:
TIMEZONE = 3 # this is for converting timestamp to UTC+0 datetime

#########
# gitcoin 07.09.2022 - 22.09.2022 
feats = ['address','grant_id','created_on','amount_in_usdt','raw_amount_in_token','token']
df1 = pd.read_csv('data/1_gitcoin_round/GR15_contributions.csv')
f1 = df1.address.map(len) == 42
f2 = df1.checkout_type.isin(['eth_zksync','eth_std','eth_polygon'])
df1 = df1[f1&f2][feats]
df1.columns = ['source_wallet','destination_wallet','created_ts','amt_usd','amt_token','token']
df1['destination_wallet'] = 'id_' + df1['destination_wallet'].astype(str)
df1['created_ts'] = df1.created_ts.map(lambda x: int(pd.to_datetime(x).timestamp()))
df1.insert(0, 'round_name', '1_gitcoin_gr15')
df = df1

#########
# unicef 09.12.2022 - 21.12.2022
feats = ['source_wallet','destination_wallet','created_at','amount','token']
map_token = {
    '0x0000000000000000000000000000000000000000': 'ETH', 
    '0x6b175474e89094c44da98b954eedeac495271d0f': 'DAI',
}
map_price = {
    'ETH': 1217.59,
    'DAI': 1,
}
df1 = pd.read_csv('data/2_unicef_round/unicef_grant_votes.csv')[feats]
df1.insert(4,'amt_token',0)
df1['token'] = df1.token.map(map_token)
df1['amt_token'] = df1['amount'].astype(float)/10**18
df1['amount'] = df1.amt_token * df1.token.map(map_price)
df1.columns = ['source_wallet','destination_wallet','created_ts','amt_usd','amt_token','token']
df1.insert(0, 'round_name', '2_unicef')
df = pd.concat([df, df1])

#########
# fantom 12.12.2022 - 02.01.2023
feats = ['source_wallet','destination_wallet','created_at','amount','token']
map_token = {
    '0x0000000000000000000000000000000000000000': 'FTM',
    '0x21be370d5312f44cb42ce377bc9b8a0cef1a4c83': 'WFTM',
    '0x8d11ec38a3eb5e956b052f67da8bdc9bef8abf3e': 'DAI',
    '0xc931f61b1534eb21d8c11b24f3f5ab2471d4ab50': 'BUSD',
}
map_price = {
    'FTM':  0.2,
    'WFTM': 0.2,
    'DAI':  1,
    'BUSD': 1,
}
df1 = pd.read_csv('data/3_fantom_round/fantom_grant_votes.csv')[feats]
df1.insert(4,'amt_token',0)
df1['token'] = df1.token.map(map_token)
df1['amt_token'] = df1['amount'].astype(float)/10**18
df1['amount'] = df1.amt_token * df1.token.map(map_price)
df1.columns = ['source_wallet','destination_wallet','created_ts','amt_usd','amt_token','token']
df1.insert(0, 'round_name', '3_fantom')
df = pd.concat([df, df1])

#########
# gitcoin alpha 17.01.2023 - 31.01.2023
map_price = {
    'ETH':  1612.72,
    'DAI':  1,
}
feats = ['source_wallet','destination_wallet','created_at','amount','token']

df1 = pd.read_csv('data/4_gitcoin_alpha/climate_grant_votes.csv')[feats]
df1.insert(4,'amt_token',0)
df1['amt_token'] = df1['amount']
df1['amount'] = df1.amt_token * df1.token.map(map_price)
df1.columns = ['source_wallet','destination_wallet','created_ts','amt_usd','amt_token','token']
df1['created_ts'] = df1.created_ts.map(lambda x: int(pd.to_datetime(x).timestamp()))
df1.insert(0, 'round_name', '41_gitcoin_climate')
df = pd.concat([df, df1])

df1 = pd.read_csv('data/4_gitcoin_alpha/ethereum_grant_votes.csv')[feats]
df1.insert(4,'amt_token',0)
df1['amt_token'] = df1['amount']
df1['amount'] = df1.amt_token * df1.token.map(map_price)
df1.columns = ['source_wallet','destination_wallet','created_ts','amt_usd','amt_token','token']
df1['created_ts'] = df1.created_ts.map(lambda x: int(pd.to_datetime(x).timestamp()))
df1.insert(0, 'round_name', '42_gitcoin_ethereum')
df = pd.concat([df, df1])

df1 = pd.read_csv('data/4_gitcoin_alpha/oss_grant_votes.csv')[feats]
df1.insert(4,'amt_token',0)
df1['amt_token'] = df1['amount']
df1['amount'] = df1.amt_token * df1.token.map(map_price)
df1.columns = ['source_wallet','destination_wallet','created_ts','amt_usd','amt_token','token']
df1['created_ts'] = df1.created_ts.map(lambda x: int(pd.to_datetime(x).timestamp()))
df1.insert(0, 'round_name', '43_gitcoin_oss')
df = pd.concat([df, df1])


#########
# save to files
df['source_wallet'] = df['source_wallet'].str.lower()
df['destination_wallet'] = df['destination_wallet'].str.lower()
df.insert(4, 'created_dt', '')
df['created_dt'] = df.created_ts.map(lambda x: dt.fromtimestamp(x-TIMEZONE*3600))
df.to_csv('data/data_rounds.csv', index = False)
print(df.shape)
display(df[:3])

df.groupby('round_name').size()

(753275, 8)


,round_name,source_wallet,destination_wallet,created_ts,created_dt,amt_usd,amt_token,token
0,1_gitcoin_gr15,0x76f69dcddd0593b0aff5fd3280c3433ddb68e0d2,id_12,1663891197,2022-09-22 23:59:57,0.06,0.00,ETH
1,1_gitcoin_gr15,0x76f69dcddd0593b0aff5fd3280c3433ddb68e0d2,id_6972,1663891195,2022-09-22 23:59:55,1.28,0.00,ETH
2,1_gitcoin_gr15,0x1bc5ebee4738fd95bd96751c45a90889f772e0f3,id_12,1663891193,2022-09-22 23:59:53,1.35,0.00,ETH


round_name
1_gitcoin_gr15         453861
2_unicef                64180
3_fantom               139337
41_gitcoin_climate      10462
42_gitcoin_ethereum     12022
43_gitcoin_oss          73413
dtype: int64

In [2]:
# df[:100]